In [1]:
import re
import os
import patoolib
import numpy as np
import pandas as pd

# Original Dataset

In [2]:
# This dataset was extracted from ClinicalTrials.gov on 15.02.24.
# Original .csv file provided as a .rar due to the maximum upload size limitations on GitHub.
# With the filter 'cancer' for disease and 'Radiation Therapy' for the intervention/treatment.
# While loading, we skip 'bad' lines. 

rar_path = 'ClinicalTrials_raw_data.rar'

destination_dir = '.' 
patoolib.extract_archive(rar_path, outdir=destination_dir, verbosity = -1)

data = pd.read_csv('ClinicalTrials_raw_data.csv', on_bad_lines="skip")
data = data.drop(['Study URL', 'Study Documents', 'Last Update Posted',
                  'Results First Posted', 'Completion Date', 'Other IDs',
                  'Acronym', 'NCT Number', 'Primary Completion Date', 'First Posted',
                  'Study Results', 'Primary Outcome Measures',
                  'Secondary Outcome Measures', 'Other Outcome Measures',
                  'Sponsor', 'Collaborators', 'Brief Summary', 'Enrollment'], axis=1)

print(f' Loaded dataset with {data.shape[0]} trials.')

 Loaded dataset with 12488 trials.


# Filtering Pipeline

### I) Keep Only Trials Started Between 2003 and 2023

In [3]:
filtered_data = data[data['Start Date'].notna()].copy()
filtered_data.loc[:, 'Original Conditions'] = filtered_data['Conditions']
filtered_data['Start Date'] = filtered_data['Start Date'].astype(str)
filtered_data['Year'] = filtered_data['Start Date'].str.split('-').str[0].astype(int)
filtered_data = filtered_data[(filtered_data['Year'] >= 2003) & (filtered_data['Year'] <= 2023)]
#filtered_data = filtered_data.drop(columns=['Year'])
filtered_data['Start Date'] = filtered_data['Start Date'].str.split('-').str[0].astype(int)
print(f'Number of trials from the original dataset with start dates between January 1, 2003, and December 31, 2023: {filtered_data.shape[0]} trials.')

Number of trials from the original dataset with start dates between January 1, 2003, and December 31, 2023: 10982 trials.


### II) Keep Only Interventional Trials with Treatment as the Primary Purpose

In [4]:
filtered_data = filtered_data[filtered_data['Study Type'] == 'INTERVENTIONAL']
filtered_data = filtered_data[filtered_data['Study Design'].str.contains('Primary Purpose: TREATMENT')]
print('Number of interventional trials with "treatment" as the primary purpose '
      f'from the original dataset: {filtered_data.shape[0]} trials.')

Number of interventional trials with "treatment" as the primary purpose from the original dataset: 7887 trials.


### III) Remove Any Trials with Essential Information Missing

In [5]:
filtered_data = filtered_data[filtered_data['Phases'].notna()]
print(f'Current number of trials: {filtered_data.shape[0]}')

Current number of trials: 6151


In [6]:
filtered_data = filtered_data[filtered_data['Sex'].notna()]
print(f'Current number of trials: {filtered_data.shape[0]}')

Current number of trials: 6150


In [7]:
filtered_data = filtered_data[filtered_data['Age'].notna()]
print(f'Current number of trials: {filtered_data.shape[0]}')

Current number of trials: 6150


In [8]:
filtered_data = filtered_data[filtered_data['Locations'].notna()]
print(f'Current number of trials: {filtered_data.shape[0]}')

Current number of trials: 5802


In [9]:
filtered_data = filtered_data[filtered_data['Funder Type'] != 'UNKNOWN']
print(f'Current number of trials: {filtered_data.shape[0]}')

Current number of trials: 5795


### IV) Filter out all non-radiation oncology trials

In [10]:
# Case-insensitive function to find specific words in text strings.

def find(word, text):
    return len(re.findall(r'{}' .format(word), text, re.IGNORECASE))!=0

In [11]:
# Lists `rt_list` and `exact_rt_list` were defined through the iterative process
# of investigating the 'Intervention/Treatment' fields. 
# This allows for the proper sorting of only the radiation oncology trials (trials that performed radiotherapy).
# The 'Intervention/Treatment' field is used because, according to ClinicalTrials.gov,
# intervention/treatment refers to any process or action that's the focus of a clinical study. 
# This includes drugs, medical devices, procedures, vaccines, and other products, whether investigational or available. 
# Thus, if radiotherapy was involved in the trial, it should be mentioned in 'Intervention/Treatment'.

rt_list = ['radiation', 'radiotherapy', 'radiosurgery', 'brachy', 'tomotherapy', 'radiodynamic',
              'hypofraction', 'Radiochemo', 'fractionated', 'fractionation', 'conv', 'stereotactic',
              'external beam', 'cyberknife', 'simultaneous integrated boost', 'simultaneous boost',
              'IMRT', 'VMAT', 'SBRT', 'SABR', 'BNCT', 'IMPT', 'SRS', 'PBI', 'PRDR', 'TBI', 'TLI',
              'TSEB', 'HDR', 'PBT', 'SIB', 'WBRT', 'CCRT',
              'volume modulated arc therapy', 'pencil beam scanning', 
              'particle therapy', 'proton', 'carbon', 'electron', 'photon', 'gray']

exact_rt_list = ['Gy', 'GY', 'RT', 'EBR', 'radio(-chemo)therapy']

for index, _ in filtered_data.iterrows():
    intervention = filtered_data['Interventions'][index]
    condition = filtered_data['Conditions'][index]
    title = filtered_data['Study Title'][index]
    
    if 'RADIATION:' in intervention:
        list_of_interventions = []
        for part in intervention.split('|'):
            if 'RADIATION:' in part:
                list_of_interventions.append(part[10:])
        
        switch = False
        for intervention in list_of_interventions:
            
            for rt_list_instance in rt_list + ['dose']:
                if find(rt_list_instance, intervention):
                    switch = True
            
            for exact_rt_list_instance in exact_rt_list: 
                if exact_rt_list_instance in intervention:
                    switch = True
                    
        if switch == False:
            filtered_data = filtered_data.drop(index)
            
    else:
        switch = False
        for rt_list_instance in rt_list:
            if find(rt_list_instance, intervention):
                switch = True

        for exact_rt_list_instance in exact_rt_list: 
            if exact_rt_list_instance in intervention:
                switch = True

        if switch == False:
            filtered_data = filtered_data.drop(index)

In [12]:
# At this step, all detected false-positive trials are removed. 
# Originally, some of false positives were detected not only through
# manual filtering steps but also through manual binning steps. 
# However, now we remove them as part of the filtering phase to keep the next pipeline focused solely on binning.

filtered_data = filtered_data.drop(118)
filtered_data = filtered_data.drop(388)
filtered_data = filtered_data.drop(1022)
filtered_data = filtered_data.drop(2425)
filtered_data = filtered_data.drop(7277)
filtered_data = filtered_data.drop(7642)
filtered_data = filtered_data.drop(7926)
filtered_data = filtered_data.drop(8368)
filtered_data = filtered_data.drop(8643)
filtered_data = filtered_data.drop(11581)

In [13]:
print(f'After the cleaning, we end up with {filtered_data.shape[0]} trials.')

After the cleaning, we end up with 4253 trials.


# Binning Pipeline

In [14]:
# These are all detected unresolved trials from the entire binning pipeline. 
# They were identified through manual screening during the different binning phases
# and were initially assigned to more than one bin. 
# After the manual screening, the correct bin was assigned.
#
# Based on the performed analysis,only less than 1% of trials 
# needed manual assignment/correction.

filtered_data.loc[497, 'Conditions'] = 'FILTERED_CNS'
filtered_data.loc[498, 'Conditions'] = 'FILTERED_CNS'
filtered_data.loc[790, 'Conditions'] = 'FILTERED_HeadAndNeck'
filtered_data.loc[881, 'Conditions'] = 'FILTERED_Digestive'
filtered_data.loc[1086, 'Conditions'] = 'FILTERED_Others'
filtered_data.loc[1432, 'Conditions'] = 'FILTERED_CNS'
filtered_data.loc[1831, 'Conditions'] = 'FILTERED_HeadAndNeck'
filtered_data.loc[2066, 'Conditions'] = 'FILTERED_HeadAndNeck'
filtered_data.loc[2251, 'Conditions'] = 'FILTERED_HeadAndNeck'
filtered_data.loc[2524, 'Conditions'] = 'FILTERED_Genitourinary'
filtered_data.loc[2714, 'Conditions'] = 'FILTERED_Genitourinary'
filtered_data.loc[3026, 'Conditions'] = 'FILTERED_Digestive'
filtered_data.loc[3045, 'Conditions'] = 'FILTERED_Hematology'
filtered_data.loc[3263, 'Conditions'] = 'FILTERED_Others'
filtered_data.loc[4271, 'Conditions'] = 'FILTERED_Thoracic'
filtered_data.loc[4504, 'Conditions'] = 'FILTERED_HeadAndNeck'
filtered_data.loc[4568, 'Conditions'] = 'FILTERED_Others'
filtered_data.loc[5102, 'Conditions'] = 'FILTERED_HeadAndNeck'
filtered_data.loc[5382, 'Conditions'] = 'FILTERED_CNS'
filtered_data.loc[5593, 'Conditions'] = 'FILTERED_Others'
filtered_data.loc[5722, 'Conditions'] = 'FILTERED_CNS'
filtered_data.loc[5816, 'Conditions'] = 'FILTERED_Others'
filtered_data.loc[5871, 'Conditions'] = 'FILTERED_Digestive'
filtered_data.loc[6153, 'Conditions'] = 'FILTERED_Hematology'
filtered_data.loc[6302, 'Conditions'] = 'FILTERED_Digestive'
filtered_data.loc[6375, 'Conditions'] = 'FILTERED_Digestive'
filtered_data.loc[7245, 'Conditions'] = 'FILTERED_Digestive'
filtered_data.loc[7607, 'Conditions'] = 'FILTERED_HeadAndNeck'
filtered_data.loc[7713, 'Conditions'] = 'FILTERED_HeadAndNeck'
filtered_data.loc[7594, 'Conditions'] = 'FILTERED_Digestive'
filtered_data.loc[7936, 'Conditions'] = 'FILTERED_CNS'
filtered_data.loc[8281, 'Conditions'] = 'FILTERED_HeadAndNeck'
filtered_data.loc[8285, 'Conditions'] = 'FILTERED_HeadAndNeck'
filtered_data.loc[9095, 'Conditions'] = 'FILTERED_Hematology'
filtered_data.loc[10103, 'Conditions'] = 'FILTERED_Digestive'
filtered_data.loc[10614, 'Conditions'] = 'FILTERED_Digestive'
filtered_data.loc[10615, 'Conditions'] = 'FILTERED_HeadAndNeck'
filtered_data.loc[11003, 'Conditions'] = 'FILTERED_CNS'
filtered_data.loc[11031, 'Conditions'] = 'FILTERED_CNS'
filtered_data.loc[12154, 'Conditions'] = 'FILTERED_HeadAndNeck'

### I) Find All Oligometastatic (OMD) Trials

In [15]:
# We start with OMD trials because we are not interested in specific locations,
# only in the fact that it is an OMD trial.

for index, _ in filtered_data.iterrows():
    condition = filtered_data['Conditions'][index]
    title = filtered_data['Study Title'][index]
    if (find('oligomet', title) or find('oligomet', condition) 
        or find('oligopro', title) or find('oligopro', condition)
        or find('oligorec', title) or find('oligorec', condition)
        or find('oligo-resid', title) or find('oligo-resid', condition)
       ):
        filtered_data.loc[index, 'Conditions'] = 'FILTERED_Oligometastatic'

### II) Find Trials That Belong to the "Others" Bin

In [16]:
# Next, we proceed with specific words for the "Others" bin,
# as it is easier to sort out the rest of the bins afterwards.
# The list of `filtered_others_definitions` was defined through 
# the iterative process of binning to decrease 
# the amount of manual screening in subsequent steps.
# The `filtered_others_definitions` list is specific to this dataset.

filtered_others_definitions = ['osteosarcoma', 'lentigo maligna', 'basal cell', 'rhabdomyosarcoma', 
                               'soft tissue sarcoma', 'retroperitoneal sarcoma', 'skin cancer',
                               'liposarcoma', 'angiosarcoma', 'ewing', 'merkel','retinoblastoma',
                               'neuroblastoma', 'chordoma']

for index, _ in filtered_data.iterrows():
    condition = filtered_data['Conditions'][index]
    title = filtered_data['Study Title'][index]
    for filtered_word in filtered_others_definitions:
        if 'FILTERED_' not in condition and (find(filtered_word, title) or find(filtered_word, condition)):    
            filtered_data.loc[index, 'Conditions'] = 'FILTERED_Others'

### III) Find All Metastatic Trials That Belong to the CNS Bin

In [17]:
# Filtering in such an order (starting with metastatic CNS trials) 
# helps to overcome the problem of binning into multiple bins later 
# and decreases the amount of manual screening needed at further steps. 
# Trials filtered as CNS at this step were reviewed manually. 
# If any inaccuracies were found, the correct bin was assigned by specifying it manually.

for index, _ in filtered_data.iterrows():
    condition = filtered_data['Conditions'][index]
    title = filtered_data['Study Title'][index]
    if ('FILTERED_' not in condition and not find('non-met', title) and not find('non met', title) 
        and not find('non-met', condition) and not find('non met', condition) 
        and not find('nonmet', title) and not find('nonmet', condition) 
        and (find('metasta', condition) or find('metasta', title)) 
        and (find('brain', title) or find('brain', condition) 
        or find('cns', title) or find('cns', condition)) 
        and not find('non-cns', condition) and not find('non-cns', title)):             
        filtered_data.loc[index, 'Conditions'] = 'FILTERED_CNS'

### IV) Define a filter for rest of the bins and sort them

In [18]:
# The `locations` dictionary was defined through an iterative process 
# to cover as many trials as possible without manual screening. 
# There is a chance that this dictionary may not be sufficient 
# for similar analyses on a different dataset, 
# as it is dataset-specific and was designed primarily to sort the current trials.

locations = {
    'FILTERED_Breast': ['breast', 'ductal carcinoma'],
    
    'FILTERED_Digestive': ['rect', 'intestin', 'gastr', 'pancr', 'liver', 'esophag', 'œsophag',
                           'anal', 'biliary', 'hepatocel', 'abdominal', 'cholangiocarcinoma',
                           'anus', 'stomach', 'HCC', 'colon', 'cholangiocellular carcinoma', 
                           'GIST', 'ESCC', 'hepatic metastasis'],
    
    'FILTERED_Thoracic': ['lung','thym', 'pleural', 'mediastinal', 'pancoast', 'NSCLC', 'SCLC', 
                          'mesothelioma', 'thyroid'],
    
    'FILTERED_Genitourinary': ['prostat', 'bladder', 'kidney', 'penile', 'renal', 'urothel', 'penis',
                               'testicular', 'seminoma', 'wilms tumor', 'adrenocortical carcinoma'],
    
    'FILTERED_Gynecology': ['endometr', 'ovar', 'cervi', 'vulva', 'vagin', 'pelv', 'uter', 
                            'gynecologic cancer', 'gynecological malignancies'],
    
    'FILTERED_Hematology': ['leukemia', 'leukaemia', 'lymphoma', 'myeloma', 'hemato', 'myelofibros',
                            'blood cancer', 'plasmacytoma', 'mycosis fungoides', 'spine metastases'],
    
    'FILTERED_CNS': ['brain', 'spinal', 'glio', 'cns', 'central nervous system', 'astrocytoma', 
                     'intracranial germ cell tumors', 'ependymoma', 'medulloblastoma', 'choroid plexus',
                     'intracranial germinoma', 'hemangioma of vertebral column', 'vestibular schwannoma',
                     'acoustic neuroma', 'cerebellar metastases', 'neoplasm to the spine', 
                     'leptomeningeal', 'vertebral metastasis'],
    
    'FILTERED_HeadAndNeck': ['oral', 'pharyn', 'laryn', 'salivary', 'sinonasal', 'head and neck', 
                             'Head and Neck', 'OSCC', 'HNSCC', 'nasal', 'Head & Neck', 'H&N',
                             'meningioma', 'adenoid', 'skull', 'nasopharyng', 'NPC', 'mouth neoplasms']
}

def check_category_overlap(text, locations):
    category_hits = set()
    for category, keywords in locations.items():
        for keyword in keywords:
            if find(keyword, text):
                category_hits.add(category)
    return category_hits


In [19]:
# Firstly, we bin with respect to 'conditions' because, 
# according to ClinicalTrials.gov: In the 'Condition' field, 
# authors should specify the disease, disorder, syndrome, 
# illness, or injury that is being studied. 
#
# This phase was performed without any manual screening
# and uniquely defined bins for more than 3,500 trials.

for index, row in filtered_data.iterrows():
    condition = row['Conditions']
    
    condition_categories = check_category_overlap(condition, locations)
    
    if 'FILTERED_' not in condition and len(condition_categories.union(condition_categories)) == 1:
        filtered_data.loc[index, 'Conditions'] = next(iter(condition_categories))

In [20]:
# If it was not possible to uniquely define a bin through conditions alone, 
# then for the trials where no bins were defined from conditions, 
# bins were determined from the title if bin was unique to a title.
#
# In cases where conditions led to more than one bin being defined, 
# the trial's title and conditions were manually screened, 
# with priority given to the title; 
#
# Any detected inaccuracies were corrected manually.
# After manual correction, by combining both filters, 
# we end up with the following filter, 
# which only processes the trials where the bin was not defined based on conditions 
# and is now correctly and uniquely defined based on the title. 
# With all exceptions being already corrected manually.

for index, row in filtered_data.iterrows():
    condition = row['Conditions']
    title = row['Study Title']
    
    condition_categories = check_category_overlap(condition, locations)
    title_categories = check_category_overlap(title, locations)
        
    if 'FILTERED_' not in condition and len(title_categories.union(title_categories)) == 1:
        filtered_data.loc[index, 'Conditions'] = next(iter(title_categories))

In [21]:
# Next, manual screening of trials with zero defined bins for both titles and conditions was performed. 
# As a result, some bins were mannualy corrected, while the rest were assigned to "Others" bin.

for index, row in filtered_data.iterrows():
    condition = row['Conditions']
    title = row['Study Title']
    
    condition_categories = check_category_overlap(condition, locations)
    title_categories = check_category_overlap(title, locations)
        
    if ('FILTERED_' not in condition 
        and len(condition_categories.union(condition_categories)) == 0 
        and len(title_categories.union(title_categories)) == 0):

        filtered_data.loc[index, 'Conditions'] = 'FILTERED_Others'

In [22]:
# The rest of the trials were again manually screened before assigning them to the 'Others' bin.
# If any incorrectness was found, the correct bin was assigned manually.

for index, row in filtered_data.iterrows():
    condition = row['Conditions']
    title = row['Study Title']
    
    condition_categories = check_category_overlap(condition, locations)
    title_categories = check_category_overlap(title, locations)
    
    if 'FILTERED_' not in condition:
        filtered_data.loc[index, 'Conditions'] = 'FILTERED_Others'

# Data Refining Pipeline


### I) Extraction of the Geographical Location of the Trials

In [23]:
# After the most challenging part - binning pipeline, 
# we need to make our data look more 'presentable' for the analysis phase. 
# We would start by resolving the location of the trials.
#
# The following binning completely covers all countries in our dataset
#. 
# Any inaccuracies in the assignment of a country to the continent do not represent 
# any of the authors opinons and are random.

Europe = ['Austria', 'Belarus', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 
          'Czechia', 'Denmark', 'Finland', 'France', 'Georgia', 'Germany',
          'Greece', 'Hungary', 'Ireland', 'Italy', 'Lithuania', 'Netherlands',
          'Norway', 'Poland', 'Portugal', 'Romania', 'Serbia', 'Slovenia',
          'Spain', 'Sweden', 'Switzerland', 'Czech Republic', 'United Kingdom',
          'Ukraine', 'Slovakia', 'Monaco', 'Malta', 'Luxembourg', 'Latvia',
          'Iceland', 'Estonia', 'Bosnia and Herzegovina', 'Macedonia', 'Jersey']

Africa = ['South Africa', 'Algeria', 'Burkina Faso', 'Egypt', 'Ethiopia', 'Gabon',
          'Ghana', 'Guinea', 'Kenya', 'Mali', 'Nigeria', 'Tanzania', 'Tunisia',
          'Zambia', 'Zimbabwe', 'Uganda', 'Morocco', 'Equatorial Guinea']

Asia =['Arabia', 'Armenia', 'Azerbaijan', 'Bangladesh', 'India', 'Indonesia', 
       'Iraq', 'Israel', 'Japan', 'Jordan', 'Kuwait', 'Kyrgyzstan', 'Lebanon',
       'Malaysia', 'Kazakhstan', 'Pakistan', 'Philippines', 'Singapore',
       'Taiwan', 'Thailand', 'Turkey', 'Vietnam', 'Uzbekistan', 'Syrian Arab Republic',
       'Sri Lanka', 'Saudi Arabia', 'Russian Federation', 'Myanmar', 'Mongolia', 
       'Hong Kong', 'Brunei Darussalam', 'Iran', 'Korea', 'China']

South_America = ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Uruguay', 'Peru']

North_America = ['Canada', 'Cuba', 'Guadeloupe', 'Guatemala', 'Martinique',  'Mexico',
                 'United States', 'Trinidad and Tobago', 'Puerto Rico', 'Panama', 
                 'El Salvador', 'Dominican Republic', 'Costa Rica']

Oceania = ['Australia', 'New Zealand', 'Guam']

In [24]:
# Some countries should be modified to extract correct geographical location.

filtered_data.loc[9390, 'Locations'] = filtered_data['Locations'][9390].replace('California Cancer Consortium', 'United States')
index_to_change = [6449, 9711]
for index in index_to_change:
     filtered_data.loc[index,'Locations'] = filtered_data['Locations'][index].replace('Islamic Republic of', 'Iran')
index_to_change = [6806, 6847]
for index in index_to_change:
     filtered_data.loc[index,'Locations'] = filtered_data['Locations'][index].replace('The Former Yugoslav Republic of', 'Macedonia')
for index, _ in filtered_data.iterrows():
    for location in filtered_data['Locations'][index].split('|'):
        if 'Republic of' in location:
            if location.split(',')[-2].strip() == 'Korea':
                filtered_data.loc[index,'Locations'] = filtered_data['Locations'][index].replace('Republic of', 'Korea')

In [25]:
# We specify both countries and continents involved in the studies.

filtered_data['Locations_Country'] = None 
filtered_data['Locations_Continent'] = None

for index, _ in filtered_data.iterrows():
    location = filtered_data['Locations'][index]
    list_of_countries = []
    for country in location.split('|'):
        list_of_countries.append(str(country.split(',')[-1]).strip())
    list_of_countries = np.unique(list_of_countries)
    filtered_data.loc[index, 'Locations_Country'] = ', '.join(list_of_countries)
    
for index, _ in filtered_data.iterrows():
    location = filtered_data['Locations'][index]
    list_of_countries = []
    for country in location.split('|'):
        list_of_countries.append(str(country.split(',')[-1]).strip())
    list_of_countries = np.unique(list_of_countries)
    text =''
    for unique_country_index in range(len(list_of_countries)):
        unique_country = list_of_countries[unique_country_index]
        if unique_country_index == len(list_of_countries)-1:
            if unique_country in Europe:
                if 'Europe' in text:
                    pass
                else:
                    text += 'Europe' 
            elif unique_country in Africa:
                if 'Africa' in text:
                    pass
                else:
                    text += 'Africa'
            elif unique_country in Asia:
                text += 'Asia'
            elif unique_country in South_America:
                text += 'South_America'
            elif unique_country in North_America:
                text += 'North_America'
            elif unique_country in Oceania:
                text += 'Oceania'
            else: 
                text += unique_country
        else:
            if unique_country in Europe:
                if 'Europe' in text:
                    pass
                else:
                    text += 'Europe' 
                    text += ','
            elif unique_country in Africa:
                if 'Africa' in text:
                    pass
                else:
                    text += 'Africa' 
                    text += ','
            elif unique_country in Asia:
                if 'Asia' in text:
                    pass
                else:
                    text += 'Asia' 
                    text += ','
            elif unique_country in South_America:
                if 'South_America' in text:
                    pass
                else:
                    text += 'South_America' 
                    text += ','
            elif unique_country in North_America:
                if 'North_America' in text:
                    pass
                else:
                    text += 'North_America' 
                    text += ','
            elif unique_country in Oceania:
                if 'Oceania' in text:
                    pass
                else:
                    text += 'Oceania' 
                    text += ','
            else: 
                text += unique_country  
                text += ','
    if text[-1] ==',':
        text = text[:-1]
    filtered_data.loc[index, 'Locations_Continent'] = text 

In [26]:
#Extract Age groups for each trial.

for index in filtered_data.index:
    study_age = filtered_data['Age'][index]
    filtered_data.loc[index,'Age'] = None
    try: 
        to_print = study_age.split('\xa0')[1].replace('(','')
        to_print = to_print.replace(')','')
        filtered_data.loc[index,'Age'] = to_print.strip()
    except IndexError:
        filtered_data.loc[index,'Age'] = study_age.split('\xa0')[0].strip()
replace_values = {'ADULT, OLDER_ADULT': 'Adult, Older Adult', 'CHILD, ADULT, OLDER_ADULT': 'Child, Adult, Older Adult',
                  'CHILD, ADULT': 'Child, Adult', 'OLDER_ADULT': 'Older Adult', 'ADULT':'Adult', 'CHILD':'Child'}
filtered_data['Age'] = filtered_data['Age'].replace(replace_values)

In [27]:
# Trial status was split into “ongoing” (if study status was “not yet recruiting”,
# “recruiting”, “enrolling by invitation”, “active not recruiting” or “suspended”),
# “stopped early” (if study status was “terminated” or “withdrawn”), 
# or “completed” (if study status was “completed”), and “unknown”. 

for index, _ in filtered_data.iterrows():
    study = filtered_data['Study Status'][index]
    if 'TERMINATED' in study or 'WITHDRAWN' in study:
            filtered_data.loc[index, 'Study Status'] = 'STOPPED EARLY'
for index, _ in filtered_data.iterrows():
    study = filtered_data['Study Status'][index]
    if 'NOT_YET_RECRUITING' in study or 'RECRUITING' in study or 'ENROLLING_BY_INVITATION' in study or 'ACTIVE_NOT_RECRUITING' in study or 'SUSPENDED' in study:
            filtered_data.loc[index, 'Study Status'] = 'ONGOING'
replace_values = {'ONGOING': 'Ongoing', 'STOPPED EARLY': 'Stopped Early',
                  'UNKNOWN': 'Unknown', 'COMPLETED': 'Completed'}
filtered_data['Study Status'] = filtered_data['Study Status'].replace(replace_values)

In [28]:
# Funding source was classified into “National Institutes of Health (NIH)”,
# “industry” and “others” based on the variable funder type. 
# For trials, where the funder type did not clearly state either NIH or industry,
# the others category was assigned. 

for index, _ in filtered_data.iterrows():
    study = filtered_data['Funder Type'][index]
    filtered_data.loc[index, 'Funder Type'] =filtered_data['Funder Type'][index].replace('OTHER_GOV', 'OTHER')
    filtered_data.loc[index, 'Funder Type'] =filtered_data['Funder Type'][index].replace('INDIV', 'OTHER')
    filtered_data.loc[index, 'Funder Type'] =filtered_data['Funder Type'][index].replace('NETWORK', 'OTHER')
    filtered_data.loc[index, 'Funder Type'] =filtered_data['Funder Type'][index].replace('FED', 'OTHER')
replace_values = {'OTHER': 'Other', 'INDUSTRY': 'Industry'}
filtered_data['Funder Type'] = filtered_data['Funder Type'].replace(replace_values)

In [29]:
for index, _ in filtered_data.iterrows():
    condition = filtered_data['Conditions'][index]
    filtered_data.loc[index,'Conditions'] = filtered_data['Conditions'][index].replace(condition, 
                                                                                       condition.split('_')[1].strip())
replace_values = {'HeadAndNeck': 'Head and Neck', 'Hematology': 'Hematology System', 'Gynecology': 'Gynecology System',
                  'CNS':'Central Nervous System'}
filtered_data['Conditions'] = filtered_data['Conditions'].replace(replace_values)

replace_values = {'MALE': 'Male', 'ALL': 'All',
                  'FEMALE': 'Female'}
filtered_data['Sex'] = filtered_data['Sex'].replace(replace_values)

replace_values = {'PHASE1': 'Phase I', 'PHASE2': 'Phase II',
                  'PHASE1|PHASE2': 'Phase I & Phase II', 'PHASE4': 'Phase IV', 'PHASE3':'Phase III',
                  'PHASE2|PHASE3':'Phase II & Phase III', 'EARLY_PHASE1': 'Early Phase I'}
filtered_data['Phases'] = filtered_data['Phases'].replace(replace_values)

In [30]:
# Remove unnescessary columns

filtered_data = filtered_data.drop(['Interventions', 'Study Type', 'Study Design',
                                    'Start Date', 'Locations', 'Original Conditions'], axis=1)

In [31]:
# Save to the Excel sheet

filtered_data.to_excel("filtered_data.xlsx", index=False)